In [1]:
## Data Collection
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import pandas as pd

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\legion\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [2]:
## load the dataset
data=gutenberg.raw('shakespeare-hamlet.txt')
## save to a file
with open('hamlet.txt','w') as file:
    file.write(data) 

In [3]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [4]:
## load the dataset
with open('hamlet.txt','r') as file:
    text=file.read().lower()

In [5]:
## Tokenize the text-creating indexes for words
tokenizer=Tokenizer()
tokenizer.fit_on_texts([text])
total_words=len(tokenizer.word_index)+1
total_words                                                                                                                                                 
input_sequences=[]
for line in text.split('\n'):
    token_list=tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_sequence=token_list[:i+1]
        input_sequences.append(n_gram_sequence)                                                                                           
max_sequence_len=max([len(x) for x in input_sequences])
input_sequences=np.array(pad_sequences(input_sequences,maxlen=max_sequence_len,padding='pre'))                                                                                               
import tensorflow as tf
x,y=input_sequences[:,:-1],input_sequences[:,-1]                                                                                                
y=tf.keras.utils.to_categorical(y,num_classes=total_words)                                                                            
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)                                                                
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout, LSTM

model=Sequential()
model.add(Embedding(total_words, 150, input_shape=(max_sequence_len-1,)))
model.add(GRU(200, return_sequences=True))
model.add(Dropout(0.3))
model.add(GRU(150))
model.add(Dropout(0.3))
model.add(Dense(total_words // 2, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(total_words, activation="softmax"))

c:\Users\legion\Python_bootcamp\venv\Lib\site-packages\keras\src\layers\core\embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [8]:
# # Compile the model
model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 13, 150)        │       722,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_2 (GRU)                     │ (None, 13, 200)        │       211,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 13, 200)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_3 (GRU)                     │ (None, 150)            │       158,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 150)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2409)           │       363,759 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 2409)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4818)           │    11,611,380 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,067,439 (49.85 MB)

 Trainable params: 13,067,439 (49.85 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
## Train the model
history=model.fit(x_train, y_train, epochs=50, validation_data=(x_test, y_test), verbose=1)

Epoch 1/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 77s 120ms/step - accuracy: 0.0570 - loss: 6.1562 - val_accuracy: 0.0635 - val_loss: 6.7570
Epoch 2/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 78s 121ms/step - accuracy: 0.0719 - loss: 5.7763 - val_accuracy: 0.0684 - val_loss: 6.9317
Epoch 3/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 77s 120ms/step - accuracy: 0.0848 - loss: 5.4992 - val_accuracy: 0.0709 - val_loss: 7.4052
Epoch 4/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 77s 119ms/step - accuracy: 0.0974 - loss: 5.2409 - val_accuracy: 0.0715 - val_loss: 7.6750
Epoch 5/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 77s 119ms/step - accuracy: 0.1084 - loss: 5.0197 - val_accuracy: 0.0734 - val_loss: 8.0194
Epoch 6/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 77s 120ms/step - accuracy: 0.1275 - loss: 4.7747 - val_accuracy: 0.0705 - val_loss: 8.2411
Epoch 7/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 77s 120ms/step - accuracy: 0.1418 - loss: 4.5999 - val_accuracy: 0.0711 - val_loss: 8.8933
Epoch 8/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 77s 120ms/step - accuracy: 0.1609 - loss: 4

In [11]:
# Function to predict the next word
def predict_next_word(model, tokenizer, text, max_sequence_len):
    token_list = tokenizer.texts_to_sequences([text])[0]                                 
    if len(token_list) >= max_sequence_len:
        token_list = token_list[-(max_sequence_len-1):]  # Ensure the sequence length matches max_sequence_len-1
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    predicted_word_index = np.argmax(predicted, axis=1)
    for word, index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word
    return None

In [13]:
input_text="To be or not to be"
print(f"Input text:{input_text}")
max_sequence_len=model.input_shape[1]+1
next_word=predict_next_word(model,tokenizer,input_text,max_sequence_len)
print(f"Next Word Prediction: {next_word}")

Input text:To be or not to be
Next Word Prediction: damn'd


In [14]:
## Save the model
model.save("next_word_GRU.h5")

In [15]:
## Save the tokenizer
import pickle
with open('tokenizer.pickle','wb') as handle:
    pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)

In [16]:
input_text="  Barn. Last night of all,When yond same"
print(f"Input text: {input_text}")
next_word=predict_next_word(model,tokenizer,input_text,max_sequence_len)
print(f"Next Word Prediction: {next_word}")

Input text:   Barn. Last night of all,When yond same
Next Word Prediction: starre
